In [1]:
import os
from glob import glob
from shutil import copy2
from string import Template

from pymatgen.io.vasp import Kpoints, Vasprun

from pymatgen.io.vasp.sets import MITMDSet

import warnings
warnings.filterwarnings("ignore")

In [2]:
vasprun_path = '/home/pzp22/Dropbox/project_/CATMAT/proj040/calculations/non_aiida/for_Ben/LiScCl_relaxation_for_Ben/8d840e1f-04a5-4950-88ec-06ce3fa01c7d/vasprun.xml'

vrun = Vasprun(vasprun_path)

strc_initial = vrun.initial_structure
strc_relaxed = vrun.final_structure




In [3]:
def get_inputs(structure, start_temp, end_temp, nsteps, time_step):
    custom_setting = {
        'ALGO': 'Fast',
        'EDIFF': 1E-5,
        'ISIF': 2,
        'ISYM': 0,
        'ADDGRID': True,
        'LASPH': True,
        'GGA': 'PS',
        'ENCUT': 500,
        'LWAVE': False,
        'NBLOCK': 1,
        'SMASS': None,
        'NELM': 200,
        'NPAR': 6,
        'LREAL': 'Auto',
        'LORBIT': None,
        'ICHARG': None,
        'LPLANE': None,
        'PREC': 'Normal',
        'LSCALU': None,
        'BMIX': None,
        'NSIM': None,
        'KBLOCK': None,
        'MDALGO': 1,
        'ANDERSEN_PROB': 0        
    }
    custom_kpoints = Kpoints(kpts=((1,1,1),))
    
    custom_potcar_settings = {
        'Li': 'Li_sv',
        'Sc': 'Sc_sv',
        'Cl': 'Cl'
    }

    inputs = MITMDSet(
        structure=structure,
        start_temp=start_temp,
        end_temp=end_temp,
        nsteps=nsteps,
        time_step=time_step,
        spin_polarized=False, 
        user_incar_settings=custom_setting, 
        user_kpoints_settings=custom_kpoints,
        user_potcar_settings=custom_potcar_settings,
        user_potcar_functional='PBE_54'
        )
    return inputs, custom_kpoints

def write_inputs(path, label, inputs, kpoints):
    tmppath = './submit_michael.sh'
    if not os.path.exists(path):
        os.makedirs(path)
    inputs.write_input(path)
    kpoints.write_file(os.path.join(path, 'KPOINTS'))

    params = {
        'job_name': f'{label}',
        'ncores': 96,
        'project_code': 'FARADAY_MSM_bjm',
        'hours': 24,
        'node_type': 'K',
        'queue_type': 'Gold',
        'vasp_bin': '/path/to/your/binary/vasp_gam'

    }
    
    with open(tmppath) as template:
        lines = template.read()
    inp = Template(lines).safe_substitute(params)
    with open(os.path.join(path,'submit.sh'), mode='w') as fobj:
        fobj.write(inp)




In [4]:
curr_dir = os.getcwd()
temperature = [400, 600]

for t in temperature:
    p = os.path.join('MD_NVE', f'{t}K')
    inputs, kpoints = get_inputs(strc_relaxed, t, t, 10000, 2) #20 ps
    write_inputs(p, f'rlx_{t}', inputs, kpoints)